### FW07 & JDRTなどのphone処理不要のデータセット作成

In [ ]:
mount='/home/hattori/dataset' # マウントポイント（絶対パスで）
exp_name='0731' # 実験名
n=0.2 #検証データ割合
exp_dir='./'+exp_name+'/datasets'

In [ ]:
# FW07
fw07_csv=mount+'/FW07/fw07_jdrt_5noise/fw07_jdrt5noise.csv'

# JDRT
## noisy05_sp7_5N
noisy05_sp7_5N_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_noisy05_sp7_5N.csv'
## clean05_sp7
clean05_sp7_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_clean05_sp7.csv'
## clean
clean_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_clean.csv'
## clean05
clean05_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_clean05.csv'
## noisy05_5N
noisy05_5N_csv=mount+'/jdrt_kaga_ver1/jdrt_kaga_noisy05_5N.csv'

readme=mount+'/readme.txt'
!cat $readme

In [ ]:
import numpy as np
import pandas as pd
import datasets,warnings
warnings.filterwarnings('ignore')

In [ ]:
#rename path
def renamepath(batch):
    batch['path']=mount+str(batch['path'])[1:]
    return batch

### Train

In [ ]:
#based_dataset & rename path
fw07=datasets.load_dataset("csv",data_files={"train":[fw07_csv]},usecols=['path','phone','noise'])
jdrt=datasets.load_dataset("csv",data_files={"train":[clean05_sp7_csv]},usecols=['path','phone','noise'])
jdrt2=datasets.load_dataset("csv",data_files={"train":[noisy05_5N_csv]},usecols=['path','phone','noise'])
sets=[jdrt,fw07]

In [ ]:
for s in sets: s['train']=s['train'].map(renamepath)

In [ ]:
#filter
jdrt2=jdrt2.filter(lambda example: example["noise"]!=0)
fw07=fw07.filter(lambda example: example["noise"]!=0)

In [ ]:
jdrt2

In [ ]:
#concat
train=datasets.concatenate_datasets([jdrt["train"],fw07['train']])

In [ ]:
#split
train=train.train_test_split(test_size=n,shuffle=True)
train

In [ ]:
#save datasets
train['train'].to_csv(exp_dir+'/train_phone.csv',columns=['path','phone'])
train['test'].to_csv(exp_dir+'/val_phone.csv',columns=['path','phone'])

In [ ]:
#cashe clear
for s in sets: s.cleanup_cache_files()

### Test

In [ ]:
# fw07 test dataset no noise
jdrt=datasets.load_dataset("csv",data_files={"test":[noisy05_5N_csv]},usecols=['path','phone'])
jdrt['test']=jdrt['test'].map(renamepath)
jdrt['test'].to_csv(exp_dir+'/test_jdrt_phone.csv',columns=['path','phone'])

In [ ]:
# fw07 test dataset no noise
fw07=datasets.load_dataset("csv",data_files={"test":[fw07_csv]},usecols=['path','phone','noise'])
fw07['test']=fw07['test'].map(renamepath)
fw07=fw07.filter(lambda example: example["noise"]==0)
fw07['test'].to_csv(exp_dir+'/test_fw07_phone.csv',columns=['path','phone'])